# Ugesco : enrich and desambiguate a sample of the Samnang-Hans' JSON

In [1]:
from ugesco import *

D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, fa

In [2]:
#test sur un Json contenant 20 éléments pris au hasard
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\jsondata_ugesco.json", "r", encoding="utf8") as file:
    json = json.load(file)
    
sample = random.sample(json, k=20)


In [3]:
data = pd.DataFrame([flatten(line) for line in sample])

In [4]:
data.head()

,beeldid,imageclassification_class1,imageclassification_class2,imageclassification_class3,imageclassification_class4,imageclassification_class5,imageclassification_prob1,imageclassification_prob2,imageclassification_prob3,imageclassification_prob4,...,spatial_key_1,spatial_key_2,spatial_value_0,spatial_value_1,spatial_value_2,temporal,temporal_key_0,temporal_key_1,temporal_value_0,temporal_value_1
0,00036913,plaza,cemetery,stage/outdoor,army_base,street,0.570686,0.081740,0.059212,0.057309,...,LOC_PLACE,NaN,Etterbeek,Brabant,NaN,NaN,NaN,NaN,NaN,NaN
1,00007485,street,slum,medina,alley,building_facade,0.666662,0.064632,0.046037,0.029069,...,LOC_PLACE,NaN,le front de l' Est,Flandres,NaN,NaN,NaN,NaN,NaN,NaN
2,00039618,street,racecourse,highway,museum/indoor,raceway,0.432363,0.140215,0.110749,0.059646,...,LOC_PLACE,LOC_PLACE,Bruxelles,Paris,Bruxelles,NaN,TIMEX_DATE,NaN,3 . IV.1932,NaN
3,00005613,street,plaza,crosswalk,fire_escape,downtown,0.665588,0.059846,0.059613,0.029834,...,NaN,NaN,"Kiosque "" De Illustratie """,NaN,NaN,NaN,TIMEX_DATE,NaN,20/3/1944,NaN
4,00011520,apartment_building/outdoor,building_facade,fire_escape,office_building,hospital,0.250683,0.153516,0.130025,0.102740,...,NaN,NaN,Anvers,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# exclude image classifications and temporal values for the moment
data = data[data.columns.drop(list(data.filter(regex='imageclassification|temporal')))]

In [6]:
# reshape the dataframe verticaly
data = data.set_index('beeldid')
data.columns = data.columns.str.split('_', expand=True)

In [7]:
data = data.stack().reset_index(level=1, drop=True).reset_index()

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
data.head()

beeldid             spatial                               
                          value        key               value
0  00036913           Etterbeek  LOC_PLACE           Etterbeek
1  00036913             Brabant  LOC_PLACE             Brabant
2  00007485  le front de l' Est   LOC_MISC  le front de l' Est
3  00007485            Flandres  LOC_PLACE            Flandres
4  00039618           Bruxelles  LOC_PLACE           Bruxelles

In [9]:
data.columns = ['_'.join(col) for col in data.columns]

In [10]:
data.rename(columns={'beeldid_': 'beeldid'}, inplace=True)

In [11]:
data.head()

,beeldid,spatial_value,spatial_key,spatial_value
0,00036913,Etterbeek,LOC_PLACE,Etterbeek
1,00036913,Brabant,LOC_PLACE,Brabant
2,00007485,le front de l' Est,LOC_MISC,le front de l' Est
3,00007485,Flandres,LOC_PLACE,Flandres
4,00039618,Bruxelles,LOC_PLACE,Bruxelles


In [12]:
#join/merge with phototheque_pallas to get the locations. 
#The csv is gzipped to get around the 100MB file weigth limitation of Github
phototeque = pd.read_csv("data/phototheque_pallas.csv.gz", compression="gzip", encoding="utf8", dtype={'beeldid': str, 'trfwnumm': str})

data = pd.merge(data, phototeque, how='left', on=['beeldid'], suffixes=['', '_x'])

In [13]:
# Pictures that contains more than a loc in the thesaurus descriptors

#data[data['loc_qid'].str.contains(",", na = False)]

In [14]:
data.head()

,beeldid,spatial_value,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name
0,00036913,Etterbeek,LOC_PLACE,Etterbeek,http://pallas.cegesoma.be/pls/opac/opac.search...,Le centenaire de la naissance du célèbre sculp...,"La reine Astrid, [...-1935]. - 208 photos",2951,NaN,NaN,NaN,NaN,NaN,NaN
1,00036913,Brabant,LOC_PLACE,Brabant,http://pallas.cegesoma.be/pls/opac/opac.search...,Le centenaire de la naissance du célèbre sculp...,"La reine Astrid, [...-1935]. - 208 photos",2951,NaN,NaN,NaN,NaN,NaN,NaN
2,00007485,le front de l' Est,LOC_MISC,le front de l' Est,http://pallas.cegesoma.be/pls/opac/opac.search...,Départ des volontaires flamands pour le front ...,"Raf van Hulse, dirigeant SS a.i. pour la Fland...",2829,NaN,NaN,NaN,NaN,NaN,NaN
3,00007485,Flandres,LOC_PLACE,Flandres,http://pallas.cegesoma.be/pls/opac/opac.search...,Départ des volontaires flamands pour le front ...,"Raf van Hulse, dirigeant SS a.i. pour la Fland...",2829,NaN,NaN,NaN,NaN,NaN,NaN
4,00039618,Bruxelles,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,La course cycliste Paris-Bruxelles. Le Belge J...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN


In [15]:
# Merge with NER_classes (spatial_keys matched with wikidata)
ner_classes = pd.read_csv("data/ner_classes.csv")

data = pd.merge(data, ner_classes, how='left', left_on = ['spatial_key'], right_on = ['ner_class'])

In [16]:
data.head()

,beeldid,spatial_value,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class,wiki_qid,wiki_class
0,00036913,Etterbeek,LOC_PLACE,Etterbeek,http://pallas.cegesoma.be/pls/opac/opac.search...,Le centenaire de la naissance du célèbre sculp...,"La reine Astrid, [...-1935]. - 208 photos",2951,NaN,NaN,NaN,NaN,NaN,NaN,LOC_PLACE,Q56061,administrative territorial entity
1,00036913,Brabant,LOC_PLACE,Brabant,http://pallas.cegesoma.be/pls/opac/opac.search...,Le centenaire de la naissance du célèbre sculp...,"La reine Astrid, [...-1935]. - 208 photos",2951,NaN,NaN,NaN,NaN,NaN,NaN,LOC_PLACE,Q56061,administrative territorial entity
2,00007485,le front de l' Est,LOC_MISC,le front de l' Est,http://pallas.cegesoma.be/pls/opac/opac.search...,Départ des volontaires flamands pour le front ...,"Raf van Hulse, dirigeant SS a.i. pour la Fland...",2829,NaN,NaN,NaN,NaN,NaN,NaN,LOC_MISC,Q2221906,geographic location
3,00007485,Flandres,LOC_PLACE,Flandres,http://pallas.cegesoma.be/pls/opac/opac.search...,Départ des volontaires flamands pour le front ...,"Raf van Hulse, dirigeant SS a.i. pour la Fland...",2829,NaN,NaN,NaN,NaN,NaN,NaN,LOC_PLACE,Q56061,administrative territorial entity
4,00039618,Bruxelles,LOC_PLACE,Bruxelles,http://pallas.cegesoma.be/pls/opac/opac.search...,La course cycliste Paris-Bruxelles. Le Belge J...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,LOC_PLACE,Q56061,administrative territorial entity


In [17]:
# rename the new columns merged
data.rename(columns={'wiki_qid': 'ner_class_qid', 'wiki_class':'ner_class_name'}, inplace=True)

In [18]:
#remove duplicate columns
data = data.T.drop_duplicates().T

In [19]:
data.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00036913,Etterbeek,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le centenaire de la naissance du célèbre sculp...,"La reine Astrid, [...-1935]. - 208 photos",2951,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
1,00036913,Brabant,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le centenaire de la naissance du célèbre sculp...,"La reine Astrid, [...-1935]. - 208 photos",2951,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
2,00007485,le front de l' Est,LOC_MISC,http://pallas.cegesoma.be/pls/opac/opac.search...,Départ des volontaires flamands pour le front ...,"Raf van Hulse, dirigeant SS a.i. pour la Fland...",2829,NaN,NaN,NaN,NaN,NaN,NaN,Q2221906,geographic location
3,00007485,Flandres,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Départ des volontaires flamands pour le front ...,"Raf van Hulse, dirigeant SS a.i. pour la Fland...",2829,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
4,00039618,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,La course cycliste Paris-Bruxelles. Le Belge J...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity


In [20]:
# remove articles in spatial_value
data['spatial_value'] = data['spatial_value'].str.replace(pat=r"\b(le|la|l\s+'|l'|les)\b", repl="", n=1, case=False)

In [21]:
#data.to_csv(r"C:\Users\ettor\Desktop\data_ugesco.csv", header=True, index=False, encoding="utf-8")

In [22]:
data.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00036913,Etterbeek,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le centenaire de la naissance du célèbre sculp...,"La reine Astrid, [...-1935]. - 208 photos",2951,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
1,00036913,Brabant,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le centenaire de la naissance du célèbre sculp...,"La reine Astrid, [...-1935]. - 208 photos",2951,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
2,00007485,front de l' Est,LOC_MISC,http://pallas.cegesoma.be/pls/opac/opac.search...,Départ des volontaires flamands pour le front ...,"Raf van Hulse, dirigeant SS a.i. pour la Fland...",2829,NaN,NaN,NaN,NaN,NaN,NaN,Q2221906,geographic location
3,00007485,Flandres,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Départ des volontaires flamands pour le front ...,"Raf van Hulse, dirigeant SS a.i. pour la Fland...",2829,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
4,00039618,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,La course cycliste Paris-Bruxelles. Le Belge J...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity


In [23]:
#normalize spatial_value
data['spatial_value'] = data['spatial_value'].apply(lambda x: Fingerprinter(x).get_fingerprint_nonsorted())

In [24]:
#transformer les loc_place pays en loc_country
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\countries.csv", 'r', encoding="utf8") as f:
    countries = set([Fingerprinter(country).get_fingerprint_nonsorted() for country in f])

# Grab DataFrame rows where column value is present in a list
data.loc[(data['spatial_value'].isin(countries)), ['spatial_key']] = "LOC_COUNTRY"

In [25]:
data

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00036913,etterbeek,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le centenaire de la naissance du célèbre sculp...,"La reine Astrid, [...-1935]. - 208 photos",2951,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
1,00036913,brabant,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le centenaire de la naissance du célèbre sculp...,"La reine Astrid, [...-1935]. - 208 photos",2951,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
2,00007485,front de est,LOC_MISC,http://pallas.cegesoma.be/pls/opac/opac.search...,Départ des volontaires flamands pour le front ...,"Raf van Hulse, dirigeant SS a.i. pour la Fland...",2829,NaN,NaN,NaN,NaN,NaN,NaN,Q2221906,geographic location
3,00007485,flandres,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Départ des volontaires flamands pour le front ...,"Raf van Hulse, dirigeant SS a.i. pour la Fland...",2829,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
4,00039618,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,La course cycliste Paris-Bruxelles. Le Belge J...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
5,00039618,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,La course cycliste Paris-Bruxelles. Le Belge J...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
6,00039618,paris,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,La course cycliste Paris-Bruxelles. Le Belge J...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
7,00039618,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,La course cycliste Paris-Bruxelles. Le Belge J...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
8,00005613,kiosque de illustratie,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,"Kiosque ""De Illustratie"", 20/3/1944. [Sipho]",Affiches et kiosques à journaux dans les rues ...,206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
9,00011520,anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Bombardement d'Anvers : 5/4/43. [Sipho],Funérailles de victimes d'un bombardement alli...,770,sepulture--antwerpen (belgique),Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity


In [26]:
gb = data.groupby(['beeldid'])
gb.head() 

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00036913,etterbeek,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le centenaire de la naissance du célèbre sculp...,"La reine Astrid, [...-1935]. - 208 photos",2951,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
1,00036913,brabant,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le centenaire de la naissance du célèbre sculp...,"La reine Astrid, [...-1935]. - 208 photos",2951,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
2,00007485,front de est,LOC_MISC,http://pallas.cegesoma.be/pls/opac/opac.search...,Départ des volontaires flamands pour le front ...,"Raf van Hulse, dirigeant SS a.i. pour la Fland...",2829,NaN,NaN,NaN,NaN,NaN,NaN,Q2221906,geographic location
3,00007485,flandres,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Départ des volontaires flamands pour le front ...,"Raf van Hulse, dirigeant SS a.i. pour la Fland...",2829,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
4,00039618,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,La course cycliste Paris-Bruxelles. Le Belge J...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
5,00039618,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,La course cycliste Paris-Bruxelles. Le Belge J...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
6,00039618,paris,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,La course cycliste Paris-Bruxelles. Le Belge J...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
7,00039618,bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,La course cycliste Paris-Bruxelles. Le Belge J...,"Courses cyclistes, [...-1939]. - 160 photos",3214,courses cyclistes--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
8,00005613,kiosque de illustratie,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,"Kiosque ""De Illustratie"", 20/3/1944. [Sipho]",Affiches et kiosques à journaux dans les rues ...,206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
9,00011520,anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Bombardement d'Anvers : 5/4/43. [Sipho],Funérailles de victimes d'un bombardement alli...,770,sepulture--antwerpen (belgique),Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity


In [27]:
#Sélectionner les records qui contiennent moins de 3 loc_place (attention : brussels et bruxelles = 1 en fait. D'où, deux semble un bon compromis)
df_test = gb['spatial_key'].apply(lambda x: (x=='LOC_PLACE').sum()).reset_index(name='count_places')
df_test

,beeldid,count_places
0,00004362,1
1,00005613,0
2,00007485,1
3,00007921,0
4,00009500,1
5,00009736,3
6,00010687,1
7,00010703,1
8,00011520,1
9,00016068,1


In [28]:
filtre = list(df_test[df_test['count_places'] < 3]['beeldid'])  
gb_filtered = data[data['beeldid'].isin(filtre)].groupby('beeldid')

In [29]:
gb_filtered.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_code,country_name,loc_qid,loc_name,ner_class_qid,ner_class_name
0,00036913,etterbeek,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le centenaire de la naissance du célèbre sculp...,"La reine Astrid, [...-1935]. - 208 photos",2951,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
1,00036913,brabant,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le centenaire de la naissance du célèbre sculp...,"La reine Astrid, [...-1935]. - 208 photos",2951,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
2,00007485,front de est,LOC_MISC,http://pallas.cegesoma.be/pls/opac/opac.search...,Départ des volontaires flamands pour le front ...,"Raf van Hulse, dirigeant SS a.i. pour la Fland...",2829,NaN,NaN,NaN,NaN,NaN,NaN,Q2221906,geographic location
3,00007485,flandres,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Départ des volontaires flamands pour le front ...,"Raf van Hulse, dirigeant SS a.i. pour la Fland...",2829,NaN,NaN,NaN,NaN,NaN,NaN,Q56061,administrative territorial entity
8,00005613,kiosque de illustratie,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,"Kiosque ""De Illustratie"", 20/3/1944. [Sipho]",Affiches et kiosques à journaux dans les rues ...,206,vie pratique--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q41176,building
9,00011520,anvers,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Bombardement d'Anvers : 5/4/43. [Sipho],Funérailles de victimes d'un bombardement alli...,770,sepulture--antwerpen (belgique),Q31,BE,Belgique,Q12892,Anvers,Q56061,administrative territorial entity
10,00030162,belgique,LOC_COUNTRY,http://pallas.cegesoma.be/pls/opac/opac.search...,Le Championnat de Belgique sur route le 8/8/19...,"Sport : courses cyclistes, 1941-1944. - 91 photos",1776,cyclisme--belgique,Q31,BE,Belgique,NaN,NaN,Q56061,administrative territorial entity
11,00030007,liege,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,"Bunker à Liège, photographié après la libération","Libération de Liège par les alliés, 9/1944. - ...",1756,liege (belgique)--histoire--1944 (liberation),Q31,BE,Belgique,Q3992,Liège,Q56061,administrative territorial entity
12,00036020,musee colonial de tervueren,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Le célèbre musée colonial de Tervueren. [Actua...,"Vue de Tervueren, [...-1939]. - 1 photo",56231,tervueren (belgique)--photographies,Q31,BE,Belgique,Q456544,Tervueren,Q41176,building
13,00035554,musee des arts decoratifs contemporains,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Un don de 30 millions... Tel est le prix du m...,Hôtel Empain à Bruxelles : [...-1939]. - 3 ph...,44795,batiments--bruxelles (belgique),Q31,BE,Belgique,Q239,Bruxelles,Q41176,building


In [30]:
#et maintenant appliquer la concatenation uniquement si le record ne contient pas plus de deux loc_place
gb = pd.DataFrame(gb_filtered['spatial_value'].apply(lambda x: "%s" % ' '.join(x))).reset_index()
gb.head()


,beeldid,spatial_value
0,00004362,rue royale rue des colonies bruxelles rue royale
1,00005613,kiosque de illustratie
2,00007485,front de est flandres
3,00007921,bourse
4,00009500,brussel eglise st marie


In [31]:
#comment concaténer loc_name maintenant, et accéder à country_name ?
#en attendant de trouver une solution élégante, récréer un dataset contenant ces infos

gb = gb.merge(data[["beeldid", "loc_name", "country_name"]])
gb.head()

,beeldid,spatial_value,loc_name,country_name
0,00004362,rue royale rue des colonies bruxelles rue royale,NaN,Belgique
1,00004362,rue royale rue des colonies bruxelles rue royale,NaN,Belgique
2,00004362,rue royale rue des colonies bruxelles rue royale,NaN,Belgique
3,00004362,rue royale rue des colonies bruxelles rue royale,NaN,Belgique
4,00005613,kiosque de illustratie,Bruxelles,Belgique


In [32]:
#on splitte les noms de pays multiples (n'existent plus)
#gb = tidy_split(gb.merge(data[["beeldid", "country_name", "loc_name"]]), column="country_name", sep=",", keep=False)

In [33]:
#on enlève les doublons
gb.drop_duplicates(subset=None, keep='first', inplace=True)

In [34]:
gb

,beeldid,spatial_value,loc_name,country_name
0,00004362,rue royale rue des colonies bruxelles rue royale,NaN,Belgique
4,00005613,kiosque de illustratie,Bruxelles,Belgique
5,00007485,front de est flandres,NaN,NaN
7,00007921,bourse,NaN,Belgique
8,00009500,brussel eglise st marie,Bruxelles,Belgique
10,00010687,bruxelles,Bruxelles,Belgique
11,00010703,malines,Elewijt,Belgique
12,00011520,anvers,Anvers,Belgique
13,00016068,vilvoorde,Vilvorde,Belgique
14,00017584,place rogier,Bruxelles,Belgique


In [35]:
#on normalise les lieux et pays
gb['loc_name'] = gb['loc_name'].apply(lambda x: Fingerprinter(str(x)).get_fingerprint_nonsorted()).str.replace('nan',  '')
gb['country_name'] = gb['country_name'].apply(lambda x: Fingerprinter(str(x)).get_fingerprint_nonsorted()).str.replace('nan',  '')

In [45]:
# on concatène le tout en supprimant les doublons

gb['loc_concat'] = gb['spatial_value'].fillna('').map(
    str) + ' ' + gb['loc_name'].fillna('').map(str) + ' ' + gb['country_name'].fillna('').map(str)

gb['loc_concat'] = gb['loc_concat'].apply(
    lambda x: ' '.join(pd.unique(x.split())))

gb

,beeldid,spatial_value,loc_name,country_name,loc_concat
0,00004362,rue royale rue des colonies bruxelles rue royale,,belgique,rue royale des colonies bruxelles belgique
4,00005613,kiosque de illustratie,bruxelles,belgique,kiosque de illustratie bruxelles belgique
5,00007485,front de est flandres,,,front de est flandres
7,00007921,bourse,,belgique,bourse belgique
8,00009500,brussel eglise st marie,bruxelles,belgique,brussel eglise st marie bruxelles belgique
10,00010687,bruxelles,bruxelles,belgique,bruxelles belgique
11,00010703,malines,elewijt,belgique,malines elewijt belgique
12,00011520,anvers,anvers,belgique,anvers belgique
13,00016068,vilvoorde,vilvorde,belgique,vilvoorde vilvorde belgique
14,00017584,place rogier,bruxelles,belgique,place rogier bruxelles belgique


In [54]:
gb.iat[0,4]

'rue royale des colonies bruxelles belgique'

In [55]:
#appliquer get_nominatim. ATTENTION : ajouter le pays

# Nominatim bloque les requêtes répétées
requests_cache.install_cache('nominatim_cache')

s = requests.Session()

def get_nominatim(value, countrycodes=['BE',''], limit=5, lang="fr"):
    # doc : https://wiki.openstreetmap.org/wiki/Nominatim
    url = 'http://nominatim.openstreetmap.org/'
    

    params = {'q': value,
              'format': 'jsonv2',
              'addressdetails': 1,
              'limit': limit,
              'email': 'ettorerizza@mail.com',
              'polygon_kml': 0,
              'extratags': 1,
              'namedetails': 0,
              # ajouter un country_code vide simule 
              # la péréférence pour un pays de geopy
              # sans se limiter à celui-ci
              'countrycodes': countrycodes,
              'accept-language': lang
              }

    headers = {
        'User-Agent': 'Ugesco app',
        'From': 'ettorerizza@mail.com'
    }

    result = s.get(url, params=params, headers=headers, timeout=10).json()

    # Nominatim bloque les requêtes trop rapides
    sleep(1)

    return result

gb['nominatim'] = gb['loc_concat'].apply(get_nominatim)

In [56]:
gb.head(10)

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim
0,00004362,rue royale rue des colonies bruxelles rue royale,,belgique,rue royale des colonies bruxelles belgique,[]
4,00005613,kiosque de illustratie,bruxelles,belgique,kiosque de illustratie bruxelles belgique,[]
5,00007485,front de est flandres,,,front de est flandres,[]
7,00007921,bourse,,belgique,bourse belgique,"[{'place_id': '68588319', 'licence': 'Data © O..."
8,00009500,brussel eglise st marie,bruxelles,belgique,brussel eglise st marie bruxelles belgique,[]
10,00010687,bruxelles,bruxelles,belgique,bruxelles belgique,"[{'place_id': '17872759', 'licence': 'Data © O..."
11,00010703,malines,elewijt,belgique,malines elewijt belgique,[]
12,00011520,anvers,anvers,belgique,anvers belgique,"[{'place_id': '179011988', 'licence': 'Data © ..."
13,00016068,vilvoorde,vilvorde,belgique,vilvoorde vilvorde belgique,"[{'place_id': '179924422', 'licence': 'Data © ..."
14,00017584,place rogier,bruxelles,belgique,place rogier bruxelles belgique,"[{'place_id': '225217654', 'licence': 'Data © ..."


In [57]:
#Comment attribuer un score de fiabilité aux résultats et choisir le meilleur s'il y en a un ?

def get_element(array, *args):
    """parse the first nominatim json."""
    for el in array:
        return dig(el, *args)
    
gb['main_loc'] = np.vectorize(get_element)(gb['nominatim'], "display_name")
gb['type_loc'] = np.vectorize(get_element)(gb['nominatim'], "type")
gb['city_loc'] = np.vectorize(get_element)(gb['nominatim'], "address", "city")
gb['country_loc'] = np.vectorize(get_element)(gb['nominatim'], "address", "country") 

gb 

,beeldid,spatial_value,loc_name,country_name,loc_concat,nominatim,main_loc,type_loc,city_loc,country_loc
0,00004362,rue royale rue des colonies bruxelles rue royale,,belgique,rue royale des colonies bruxelles belgique,[],None,None,None,None
4,00005613,kiosque de illustratie,bruxelles,belgique,kiosque de illustratie bruxelles belgique,[],None,None,None,None
5,00007485,front de est flandres,,,front de est flandres,[],None,None,None,None
7,00007921,bourse,,belgique,bourse belgique,"[{'place_id': '68588319', 'licence': 'Data © O...","Bourse, Rue de la Bourse, Quartier du Centre, ...",attraction,Ville de Bruxelles,Belgique
8,00009500,brussel eglise st marie,bruxelles,belgique,brussel eglise st marie bruxelles belgique,[],None,None,None,None
10,00010687,bruxelles,bruxelles,belgique,bruxelles belgique,"[{'place_id': '17872759', 'licence': 'Data © O...","Bruxelles, Bruxelles-Capitale, 1000, Belgique",city,Bruxelles,Belgique
11,00010703,malines,elewijt,belgique,malines elewijt belgique,[],None,None,None,None
12,00011520,anvers,anvers,belgique,anvers belgique,"[{'place_id': '179011988', 'licence': 'Data © ...","Anvers, Flandre, Belgique",city,Anvers,Belgique
13,00016068,vilvoorde,vilvorde,belgique,vilvoorde vilvorde belgique,"[{'place_id': '179924422', 'licence': 'Data © ...","Vilvoorde, Vilvorde, Hal-Vilvorde, Brabant fla...",administrative,None,Belgique
14,00017584,place rogier,bruxelles,belgique,place rogier bruxelles belgique,"[{'place_id': '225217654', 'licence': 'Data © ...","Rogier, Place Charles Rogier, Saint-Josse-ten-...",station,None,Belgique


In [ ]:
#en attendant de trouver un moyen de chosir le bon Nominatim, traitons avec Wikipedia les beeldid dont main_loc est None
